In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/My Drive/iprofihack/clever/train.csv" train.csv
!cp "/content/drive/My Drive/iprofihack/clever/onlyquestions" onlyquestions
!cp "/content/drive/My Drive/iprofihack/clever/questionswithmisspellingcount.csv" questionswithmisspellingcount.csv

Mounted at /content/drive


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd

seed = 42

In [7]:
raw_text = open('onlyquestions').readlines()

In [8]:
transformer = TfidfVectorizer(max_features=13000, ngram_range=(1,2))
X = transformer.fit_transform(raw_text)
# X = X.toarray()
X_test = X[30000:]
X_ = X[:30000]

In [9]:
answers_df = pd.read_csv('train.csv', sep=';', index_col=0)
answers_df

,Answer
ID,
1,0
2,1
3,0
4,0
5,0
...,...
29996,0
29997,0
29998,0


In [10]:
y = answers_df['Answer'].values

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.1, random_state=seed)

In [ ]:
model = RandomForestClassifier(random_state=seed)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
# model.score(X_valid, y_valid)
roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])

0.49395011392253263

In [ ]:
forest_ = model.predict_proba(X_valid)
forest_

array([[0.98393939, 0.01606061],
       [0.92      , 0.08      ],
       [0.98133333, 0.01866667],
       ...,
       [0.89533333, 0.10466667],
       [0.99      , 0.01      ],
       [0.322     , 0.678     ]])

## catboost

In [6]:
!pip install catboost

     |████████████████████████████████| 67.3MB 132kB/s 


In [4]:
from catboost import CatBoostClassifier

In [136]:
text_with_counts = pd.read_csv('questionswithmisspellingcount.csv', sep='\t')
text_with_counts['Question'] = text_with_counts['Question'].fillna('-')
#### добавлено после презентаций
text_with_counts['Raw'] = list(map(lambda line: line[:-1], raw_text))
text_with_counts['First_capital'] = text_with_counts['Raw'].str.match('^[А-ЯA-Z]')
text_with_counts['Last_symbol'] = text_with_counts['Raw'].str.match('.*[А-Яа-яA-Za-z\d]\?$')
text_with_counts['Count_eng_words'] = text_with_counts['Raw'].str.findall('[a-zA-Z]+').apply(len)
text_with_counts['Count_caps_words'] = text_with_counts['Raw'].str.findall('([A-Z]+)|([А-Я]+)').apply(len)
text_with_counts['Clever'] = text_with_counts['Raw'].str.contains('([Кк][Лл][Ее][Вв][Ее][Рр])')
text_with_counts['Comma_errors'] = text_with_counts['Raw'].str.contains('([a-zA-Zа-яА-Я\d] [,\.\?!])|([,\.\?!][a-zA-Zа-яА-Я\d])')
text_with_counts['Privet'] = text_with_counts['Raw'].str.contains('[Зз]дравствуйте')
text_with_counts['Money'] = text_with_counts['Raw'].str.contains('деньг')
####
cols = [col for col in text_with_counts.columns if col != 'Question']
X_cb = text_with_counts[cols].values
# X_cb = raw_text
X_cb_train, X_cb_valid, y_train, y_valid = train_test_split(X_cb[:30000], y, test_size=0.1, random_state=seed)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [126]:
# text_with_counts['Parasite'].unique().tolist()

[False, True]

In [137]:
model = CatBoostClassifier(
    learning_rate=0.02,
    iterations=4000,
    random_seed=seed, 
    task_type='GPU', 
    eval_metric='AUC',
    metric_period=4, 
    text_features=[4])
model.fit(X_cb_train, y_train)

0:	learn: 0.6249865	total: 57.8ms	remaining: 3m 51s
4:	learn: 0.7135039	total: 272ms	remaining: 3m 37s
8:	learn: 0.7238907	total: 483ms	remaining: 3m 34s
12:	learn: 0.7248000	total: 689ms	remaining: 3m 31s
16:	learn: 0.7284750	total: 900ms	remaining: 3m 30s
20:	learn: 0.7279989	total: 1.11s	remaining: 3m 31s
24:	learn: 0.7306113	total: 1.31s	remaining: 3m 28s
28:	learn: 0.7319393	total: 1.53s	remaining: 3m 29s
32:	learn: 0.7313949	total: 1.7s	remaining: 3m 24s
36:	learn: 0.7309131	total: 1.92s	remaining: 3m 25s
40:	learn: 0.7304382	total: 2.15s	remaining: 3m 27s
44:	learn: 0.7315529	total: 2.37s	remaining: 3m 28s
48:	learn: 0.7347009	total: 2.59s	remaining: 3m 28s
52:	learn: 0.7359141	total: 2.8s	remaining: 3m 28s
56:	learn: 0.7363200	total: 3.02s	remaining: 3m 28s
60:	learn: 0.7357924	total: 3.23s	remaining: 3m 28s
64:	learn: 0.7367451	total: 3.4s	remaining: 3m 26s
68:	learn: 0.7380184	total: 3.63s	remaining: 3m 26s
72:	learn: 0.7383209	total: 3.83s	remaining: 3m 25s
76:	learn: 0.7394

In [138]:
roc_auc_score(y_valid, model.predict_proba(X_cb_valid)[:, 1])

0.7679320062357595

In [132]:
catboost_ = model.predict_proba(X_cb_valid)
catboost_

array([[0.93794289, 0.06205711],
       [0.74371783, 0.25628217],
       [0.93956684, 0.06043316],
       ...,
       [0.6272611 , 0.3727389 ],
       [0.86005776, 0.13994224],
       [0.88800788, 0.11199212]])

In [133]:
X_cb_test = X_cb[30000:]
catboost__ = model.predict_proba(X_cb_test)
preds = model.predict_proba(X_cb_test)[:, 1]
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit1.csv', header=None, index=None)

## xgboost

In [ ]:
!pip install xgboost

In [50]:
from xgboost import XGBClassifier

In [54]:
model = XGBClassifier(n_estimators=1000, random_state=seed)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [55]:
roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])

0.6661434224727185

In [53]:
xgboost_ = model.predict_proba(X_valid)
xgboost_

array([[0.90867245, 0.09132756],
       [0.91081405, 0.08918595],
       [0.9150182 , 0.0849818 ],
       ...,
       [0.8736875 , 0.12631251],
       [0.8818268 , 0.11817317],
       [0.9254509 , 0.07454906]], dtype=float32)

In [ ]:
X_test = X[30000:]
xgboost__ = model.predict_proba(X_test)
preds = model.predict_proba(X_test)[:, 1]
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit2.csv', header=None, index=None)

## lightgbm (после презентации)

In [30]:
!pip install lightgbm

In [86]:
import lightgbm as lgb

In [87]:
model = lgb.LGBMClassifier(learning_rate=0.02,
                           n_estimators=1000,
                           random_state=seed,)
model.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc',
        early_stopping_rounds=200)

[1]	valid_0's auc: 0.640387	valid_0's binary_logloss: 0.331699
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.639818	valid_0's binary_logloss: 0.331142
[3]	valid_0's auc: 0.643466	valid_0's binary_logloss: 0.330564
[4]	valid_0's auc: 0.643025	valid_0's binary_logloss: 0.330076
[5]	valid_0's auc: 0.647157	valid_0's binary_logloss: 0.329573
[6]	valid_0's auc: 0.64538	valid_0's binary_logloss: 0.329105
[7]	valid_0's auc: 0.645883	valid_0's binary_logloss: 0.328705
[8]	valid_0's auc: 0.647479	valid_0's binary_logloss: 0.328207
[9]	valid_0's auc: 0.64639	valid_0's binary_logloss: 0.327836
[10]	valid_0's auc: 0.644845	valid_0's binary_logloss: 0.327491
[11]	valid_0's auc: 0.643413	valid_0's binary_logloss: 0.327172
[12]	valid_0's auc: 0.641728	valid_0's binary_logloss: 0.326898
[13]	valid_0's auc: 0.639083	valid_0's binary_logloss: 0.326634
[14]	valid_0's auc: 0.63881	valid_0's binary_logloss: 0.326372
[15]	valid_0's auc: 0.640697	valid_0's binary_logloss

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.02, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [88]:
roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])
lightboost_ = model.predict_proba(X_valid)

In [60]:
X_test = X[30000:]
lightboost__ = model.predict_proba(X_test)
preds = model.predict_proba(X_test)[:, 1]
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit4.csv', header=None, index=None)

## Ensemble

In [ ]:
ensemble = ((catboost_ + xgboost_) / 2)[:, 1]
roc_auc_score(y_valid, ensemble)

0.6937156733421275

In [ ]:
preds = ((catboost__ + xgboost__) / 2)[:, 1]
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit3.csv', header=None, index=None)

# Transformers

In [61]:
!pip install transformers
!pip install -U catalyst

     |████████████████████████████████| 2.1MB 10.8MB/s 
     |████████████████████████████████| 870kB 35.1MB/s 
     |████████████████████████████████| 3.3MB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ac479507c72863ab4573276d00e5f65844616b31c55843330af9b25b4114da18
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 471kB 10.7MB/s 
     |████████████████████████████████| 122kB 25.2MB/s 
     |████████████████████████████████| 645kB 24.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [64]:
from transformers import BertForSequenceClassification
from torch.nn import functional as F
from transformers import AdamW
from transformers import BertTokenizerFast

import random
import os

import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset

from catalyst import dl, utils
from catalyst.callbacks import CheckpointCallback
from catalyst.callbacks.scheduler import SchedulerCallback
from catalyst.callbacks import EarlyStoppingCallback
# from catalyst.callbacks.tracing import TracerCallback
from catalyst.metrics import auc, f1_score, accuracy
from catalyst.utils import set_global_seed, prepare_cudnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

set_global_seed(seed)
prepare_cudnn(True)

In [65]:
tokenizer = BertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased')

class CleverDataset(Dataset):
    def __init__(self, sentences, labels=None, mode='train'):
        self.sentences = sentences
        self.labels = labels
        self.mode = mode
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, i):
        if self.mode == 'train':
            return {'text': self.sentences[i], 'targets': self.labels[i]}
        else:
            return {'text': self.sentences[i], 'targets': 0}

In [66]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased')
model.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [70]:
sentences = raw_text
labels = y
train_sentences, valid_sentences, train_labels, valid_labels = train_test_split(sentences[:30000], labels, test_size=0.1, random_state=seed)

train_dataset = CleverDataset(train_sentences, train_labels)
valid_dataset = CleverDataset(valid_sentences, valid_labels)

In [71]:
def bert_collate_fn(batch):
    encoding = tokenizer([item['text'] for item in batch], return_tensors='pt', max_length=100, padding=True, truncation=True)
    return {'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'targets': torch.tensor([item['targets'] for item in batch])}

batch_size = 100

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=bert_collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2, collate_fn=bert_collate_fn)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader   
}

In [ ]:
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
for param in model.base_model.parameters():
    param.requires_grad = False
model.classifier.requires_grad = True
optimizer = AdamW([param for param in model.parameters() if param.requires_grad == True], lr=1e-4)
criterion = nn.CrossEntropyLoss()
epochs = 30

In [ ]:
class CustomRunner(dl.Runner):

  def handle_batch(self, batch):
    # Unpack the data. Its structure depends on your model and
    # on what you pass to `train()`.
    input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['targets']
    outputs = self.model(input_ids, attention_mask=attention_mask) # Forward pass
    
    # pass network input to state `input`
    self.batch = {"targets": labels, "logits": outputs.logits}


runner = CustomRunner()

In [ ]:
model.train()
model = model.cuda()

# runner = dl.SupervisedRunner(
#     input_key=['input_ids', 'attention_mask'], 
#     output_key="logits", 
#     target_key="targets", 
#     loss_key="loss")

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    # scheduler=scheduler,
    loaders=dataloaders,
    callbacks={
        "metric_auc": dl.AUCCallback(input_key="logits", target_key="targets"),
        "early_stopping": EarlyStoppingCallback(
            patience=5,
            loader_key='train',
            metric_key="loss",
            minimize=True,
        ),
        "criterion": dl.CriterionCallback(  # special Callback for criterion computation
            target_key="targets",              # `input_key` specifies correct labels (or `y_true`) from `runner.input` 
            input_key="logits",              # `output_key` specifies model predictions (`y_pred`) from `runner.output`
            metric_key="loss",                    # `prefix` - key to use with `runner.batch_metrics`
        ),  # alias for `runner.batch_metrics[prefix] = runner.criterion(runner.output[output_key], runner.input[input_key])`
        "optimizer": dl.OptimizerCallback(
            metric_key="loss", 
            accumulation_steps=1,
            grad_clip_params=None,
        )
        # SchedulerCallback(
        #     loader_key='train',
        #     metric_key="loss"),
    },
    valid_loader="valid",
    valid_metric="auc",
    minimize_valid_metric=False,
    num_epochs=epochs,
    verbose=True,
    load_best_on_end=True,
    seed=seed,
    logdir="/content/drive/My Drive/iprofihack/clever/logs2"
)

/usr/local/lib/python3.7/dist-packages/catalyst/core/runner.py:567: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  for k, v in self.batch_metrics.items()



train (1/30) auc: 0.4854130744934082 | auc/class_00: 0.48454803228378296 | auc/class_01: 0.48627814650535583 | loss: 0.4169556200504303 | loss/mean: 0.4169556200504303 | loss/std: 0.09331723566346185 | lr: 0.0001 | momentum: 0.9



valid (1/30) auc: 0.46624475717544556 | auc/class_00: 0.4643470346927643 | auc/class_01: 0.46814244985580444 | loss: 0.34115496277809143 | loss/mean: 0.34115496277809143 | loss/std: 0.07113090324976173 | lr: 0.0001 | momentum: 0.9
* Epoch (1/30) 



train (2/30) auc: 0.5060421228408813 | auc/class_00: 0.5068587064743042 | auc/class_01: 0.5052255988121033 | loss: 0.3423343300819397 | loss/mean: 0.3423343300819397 | loss/std: 0.06384686925269346 | lr: 0.0001 | momentum: 0.9



valid (2/30) auc: 0.47038793563842773 | auc/class_00: 0.4682863652706146 | auc/class_01: 0.47248950600624084 | loss: 0.3380485475063324 | loss/mean: 0.3380485475063324 | loss/std: 0.08061826673071654 | lr: 0.0001 | momentum: 0.9
* Epoch (2/30) 


...


train (25/30) auc: 0.6062297821044922 | auc/class_00: 0.608030378818512 | auc/class_01: 0.6044291853904724 | loss: 0.32676512002944946 | loss/mean: 0.32676512002944946 | loss/std: 0.07006252386574553 | lr: 0.0001 | momentum: 0.9



valid (25/30) auc: 0.6184176206588745 | auc/class_00: 0.623059093952179 | auc/class_01: 0.6137762069702148 | loss: 0.3245927691459656 | loss/mean: 0.3245927691459656 | loss/std: 0.07355945986055865 | lr: 0.0001 | momentum: 0.9
* Epoch (25/30) 



train (26/30) auc: 0.6074048280715942 | auc/class_00: 0.6068304181098938 | auc/class_01: 0.6079792976379395 | loss: 0.32676491141319275 | loss/mean: 0.32676491141319275 | loss/std: 0.06120946207042712 | lr: 0.0001 | momentum: 0.9



valid (26/30) auc: 0.6196420192718506 | auc/class_00: 0.6242738962173462 | auc/class_01: 0.6150102019309998 | loss: 0.3244475722312927 | loss/mean: 0.3244475722312927 | loss/std: 0.07425274075486153 | lr: 0.0001 | momentum: 0.9


KeyboardInterrupt: ignored

In [ ]:
model = model.cpu()
checkpoint_path = f'/content/drive/My Drive/iprofihack/clever/{type(model).__name__}2.pth'
torch.save(model.state_dict(), checkpoint_path)

In [90]:
checkpoint_path = f'/content/drive/My Drive/iprofihack/clever/{type(model).__name__}2.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
model = model.cuda()
targets = []
preds = []
for batch in tqdm(valid_dataloader):
    with torch.no_grad():
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['targets']

        logits = model(input_ids, attention_mask).logits

        preds.append(torch.softmax(logits.cpu(), dim=-1).numpy())
        targets.append(labels.numpy())
targets = np.hstack(targets)
preds = np.vstack(preds)[:, 1]
roc_auc_score(targets, preds)

0.7272490706319703

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = True
optimizer = AdamW([param for param in model.parameters() if param.requires_grad == True], lr=1e-5)
criterion = nn.CrossEntropyLoss()
epochs = 6

In [ ]:
model.train()
model = model.cuda()

# runner = dl.SupervisedRunner(
#     input_key=['input_ids', 'attention_mask'], 
#     output_key="logits", 
#     target_key="targets", 
#     loss_key="loss")

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    # scheduler=scheduler,
    loaders=dataloaders,
    callbacks={
        "metric_auc": dl.AUCCallback(input_key="logits", target_key="targets"),
        "early_stopping": EarlyStoppingCallback(
            patience=5,
            loader_key='train',
            metric_key="loss",
            minimize=True,
        ),
        "criterion": dl.CriterionCallback(  # special Callback for criterion computation
            target_key="targets",              # `input_key` specifies correct labels (or `y_true`) from `runner.input` 
            input_key="logits",              # `output_key` specifies model predictions (`y_pred`) from `runner.output`
            metric_key="loss",                    # `prefix` - key to use with `runner.batch_metrics`
        ),  # alias for `runner.batch_metrics[prefix] = runner.criterion(runner.output[output_key], runner.input[input_key])`
        "optimizer": dl.OptimizerCallback(
            metric_key="loss", 
            accumulation_steps=1,
            grad_clip_params=None,
        )
        # SchedulerCallback(
        #     loader_key='train',
        #     metric_key="loss"),
    },
    valid_loader="valid",
    valid_metric="auc",
    minimize_valid_metric=False,
    num_epochs=epochs,
    verbose=True,
    load_best_on_end=True,
    seed=seed,
    logdir="/content/drive/My Drive/iprofihack/clever/logs"
)

/usr/local/lib/python3.7/dist-packages/catalyst/core/runner.py:567: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  for k, v in self.batch_metrics.items()



train (1/6) auc: 0.7515475749969482 | auc/class_00: 0.7513906955718994 | auc/class_01: 0.7517043948173523 | loss: 0.2894846498966217 | loss/mean: 0.2894846498966217 | loss/std: 0.056661043787394415 | lr: 1e-05 | momentum: 0.9



valid (1/6) auc: 0.7936047315597534 | auc/class_00: 0.794194757938385 | auc/class_01: 0.7930147647857666 | loss: 0.2802436649799347 | loss/mean: 0.2802436649799347 | loss/std: 0.050781084318841624 | lr: 1e-05 | momentum: 0.9
* Epoch (1/6) 



train (2/6) auc: 0.8107576370239258 | auc/class_00: 0.8105316758155823 | auc/class_01: 0.8109836578369141 | loss: 0.26675817370414734 | loss/mean: 0.26675817370414734 | loss/std: 0.05680943438152712 | lr: 1e-05 | momentum: 0.9



valid (2/6) auc: 0.8031070828437805 | auc/class_00: 0.8039453029632568 | auc/class_01: 0.8022688627243042 | loss: 0.27200135588645935 | loss/mean: 0.27200135588645935 | loss/std: 0.05415672304806811 | lr: 1e-05 | momentum: 0.9
* Epoch (2/6) 


...


valid (4/6) auc: 0.8103818893432617 | auc/class_00: 0.8107087016105652 | auc/class_01: 0.810055136680603 | loss: 0.3076756000518799 | loss/mean: 0.3076756000518799 | loss/std: 0.061904843368154 | lr: 1e-05 | momentum: 0.9
* Epoch (4/6) 



train (5/6) auc: 0.9184769988059998 | auc/class_00: 0.9184384346008301 | auc/class_01: 0.9185155630111694 | loss: 0.19948512315750122 | loss/mean: 0.19948512315750122 | loss/std: 0.044566959579784134 | lr: 1e-05 | momentum: 0.9



valid (5/6) auc: 0.7956427335739136 | auc/class_00: 0.7961937785148621 | auc/class_01: 0.7950917482376099 | loss: 0.3257308900356293 | loss/mean: 0.3257308900356293 | loss/std: 0.0696572994896269 | lr: 1e-05 | momentum: 0.9
* Epoch (5/6) 



train (6/6) auc: 0.9426587820053101 | auc/class_00: 0.9426652193069458 | auc/class_01: 0.9426524043083191 | loss: 0.17334753274917603 | loss/mean: 0.17334753274917603 | loss/std: 0.04211521467617887 | lr: 1e-05 | momentum: 0.9



valid (6/6) auc: 0.7803597450256348 | auc/class_00: 0.7806103825569153 | auc/class_01: 0.7801091074943542 | loss: 0.3724658191204071 | loss/mean: 0.3724658191204071 | loss/std: 0.10305871751216661 | lr: 1e-05 | momentum: 0.9
* Epoch (6/6) 
Top best models:
/content/drive/My Drive/iprofihack/clever/logs/checkpoints/train.4.pth	0.8104


In [ ]:
model = model.cpu()
checkpoint_path = f'/content/drive/My Drive/iprofihack/clever/{type(model).__name__}2.pth'
torch.save(model.state_dict(), checkpoint_path)

In [67]:
checkpoint_path = f'/content/drive/My Drive/iprofihack/clever/{type(model).__name__}2.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [73]:
model = model.cuda()
targets = []
probs_ = []
for batch in tqdm(valid_dataloader):
    with torch.no_grad():
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['targets']

        logits = model(input_ids, attention_mask).logits

        probs_.append(torch.softmax(logits.cpu(), dim=-1).numpy())
        targets.append(labels.numpy())
targets = np.hstack(targets)
probs_ = np.vstack(probs_)
preds = probs_[:, 1]
roc_auc_score(targets, preds)

0.7804185154095215

In [74]:
test_sentences = sentences[30000:]
test_dataset = CleverDataset(test_sentences, mode='test')
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2, collate_fn=bert_collate_fn)

In [75]:
model = model.cuda()
targets = []
probs__ = []
for batch in tqdm(test_dataloader):
    with torch.no_grad():
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()

        logits = model(input_ids, attention_mask).logits

        probs__.append(torch.softmax(logits.cpu(), dim=-1).numpy())
probs__ = np.vstack(probs__)
preds = probs__[:, 1]
# roc_auc_score(targets, preds)

In [ ]:
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit.csv', header=None, index=None)

## CatBoost + RuBERT

In [134]:
ensemble = ((probs_ + catboost_) / 2)[:, 1]
roc_auc_score(y_valid, ensemble)

0.7925308790022785

In [135]:
preds = ((catboost__ + probs__) / 2)[:, 1]
result_list = []
for i, pred in enumerate(preds):
    result_list.append((30001+i, pred))
result = pd.DataFrame(result_list, columns=['ID', 'Question'])
result.to_csv('submit5.csv', header=None, index=None)